In [0]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DoubleType

In [0]:
from pyspark.sql.functions import when

In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "",
"fs.azure.account.oauth2.client.secret": '',
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/tanent_id/oauth2/token"}

dbutils.fs.mount(
    source = 'abfss://cwc2019container@cwc2019storageaccount.dfs.core.windows.net',
    mount_point = '/mnt/cwc2019analysis',
    extra_configs = configs
)

---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
File <command-1956835900806799>, line 7
      1 configs = {"fs.azure.account.auth.type": "OAuth",
      2 "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
      3 "fs.azure.account.oauth2.client.id": "[38;5;",
      4 "fs.azure.account.oauth2.client.secret": '[38;5;',
      5 "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/tanent_id/oauth2/token"}
----> 7 dbutils.fs.mount(
      8     source = 'abfss://cwc2019container@cwc2019storageaccount.dfs.core.windows.net',
      9     mount_point = '/mnt/cwc2019analysis',
     10     extra_configs = configs
     11 )

File /databricks/python_shell/dbruntime/dbutils.py:362, in DBUtils.FSHandler.prettify_exception_message.<locals>.f_with_exception_handling(*args, **kwargs)
    360 exc.__context__ = None
    361 

In [0]:
%fs
ls 'mnt/cwc2019analysis'

path,name,size,modificationTime
dbfs:/mnt/cwc2019analysis/raw-data/,raw-data/,0,1710239011000
dbfs:/mnt/cwc2019analysis/transformed-data/,transformed-data/,0,1710239046000


In [0]:
spark

In [0]:
batsman = spark.read.format('csv').option('header', 'true').option('infrSchema', 'true').load('/mnt/cwc2019analysis/raw-data/batsman.csv')
bowler = spark.read.format('csv').option('header', 'true').option('infrSchema', 'true').load('/mnt/cwc2019analysis/raw-data/bowler.csv')
ground_averages = spark.read.format('csv').option('header', 'true').option('infrSchema', 'true').load('/mnt/cwc2019analysis/raw-data/ground_averages.csv')
odi_match_results = spark.read.format('csv').option('header', 'true').option('infrSchema', 'true').load('/mnt/cwc2019analysis/raw-data/odi_match_results.csv')
odi_match_totals = spark.read.format('csv').option('header', 'true').option('infrSchema', 'true').load('/mnt/cwc2019analysis/raw-data/odi_match_totals.csv')
wc_players = spark.read.format('csv').option('header', 'true').option('infrSchema', 'true').load('/mnt/cwc2019analysis/raw-data/wc_players.csv')

## Batsman File

In [0]:
batsman_data = batsman.select('Player_ID', 'Batsman', 'Ground', 'Opposition', 'Runs', 'BF', '4s', '6s', 'SR')

In [0]:
columns_to_replace = ['Runs', 'BF', '4s', '6s']
for column in columns_to_replace:
    batsman_data = batsman_data.withColumn(column, when(batsman_data[column] == '-', '0').otherwise(batsman_data[column]))
batsman_data = batsman_data.withColumn('SR', when(batsman_data['SR'] == '-', '0.00').otherwise(batsman_data['SR']))

In [0]:
columns_to_change_datatype = ['Player_ID', 'Runs', 'BF', '4s', '6s']
for columns in columns_to_change_datatype:
    batsman_data = batsman_data.withColumn(columns , col(columns).cast(IntegerType()))

batsman_data = batsman_data.withColumn('SR', col('SR').cast(DoubleType()))

In [0]:
batsman_data.printSchema()

root
 |-- Player_ID: integer (nullable = true)
 |-- Batsman: string (nullable = true)
 |-- Ground: string (nullable = true)
 |-- Opposition: string (nullable = true)
 |-- Runs: integer (nullable = true)
 |-- BF: integer (nullable = true)
 |-- 4s: integer (nullable = true)
 |-- 6s: integer (nullable = true)
 |-- SR: double (nullable = true)



## Bowler File

In [0]:
bowler_data = bowler.select('Player_ID', 'Bowler', 'Ground', 'Opposition', 'Overs', 'Mdns' ,'Runs', 'Wkts', 'Econ')

In [0]:
columns_cast_to_integer = ['Player_ID','Mdns', 'Runs', 'Wkts']
for column in columns_cast_to_integer:
    bowler_data = bowler_data.withColumn(column, col(column).cast(IntegerType()))

columns_cast_to_double = ['Overs', 'Econ']
for column in columns_cast_to_double:
    bowler_data = bowler_data.withColumn(column, col(column).cast(DoubleType()))

In [0]:
bowler_data = bowler_data.filter(~col("Overs").contains("-"))

In [0]:
bowler_data.printSchema()

root
 |-- Player_ID: integer (nullable = true)
 |-- Bowler: string (nullable = true)
 |-- Ground: string (nullable = true)
 |-- Opposition: string (nullable = true)
 |-- Overs: double (nullable = true)
 |-- Mdns: integer (nullable = true)
 |-- Runs: integer (nullable = true)
 |-- Wkts: integer (nullable = true)
 |-- Econ: double (nullable = true)



In [0]:
columns_to_replace = ['Player_ID','Mdns', 'Runs', 'Wkts']
for column in columns_to_replace:
    bowler_data = bowler_data.withColumn(column, when(bowler_data[column] == '-', '0').otherwise(bowler_data[column]))

bowler_data = bowler_data.withColumn('Econ', when(bowler_data['Econ'] == '-', '0.00').otherwise(bowler_data['Econ']))

## Ground_Average File

In [0]:
ground_data = ground_averages

In [0]:
columns_to_replace = ['Mat', 'Won', 'Tied', 'NR', 'Runs', 'Wkts', 'Balls']
for column in columns_to_replace:
    ground_data = ground_data.withColumn(column, when(ground_data[column] == '-', '0').otherwise(ground_data[column]))

ground_data = ground_data.withColumn('Ave', when(ground_data['Ave'] == '-', '0.00').otherwise(ground_data['Ave']))
bowler_data = ground_data.withColumn('RPO', when(ground_data['RPO'] == '-', '0.00').otherwise(ground_data['RPO']))

In [0]:
columns_cast_to_integer = ['Mat', 'Won', 'Tied', 'NR', 'Runs', 'Wkts', 'Balls']
for column in columns_cast_to_integer:
    ground_data = ground_data.withColumn(column, col(column).cast(IntegerType()))

columns_cast_to_double = ['Ave', 'RPO']
for column in columns_cast_to_double:
    ground_data = ground_data.withColumn(column, col(column).cast(DoubleType()))

In [0]:
ground_data.printSchema()

root
 |-- Ground: string (nullable = true)
 |-- Span: string (nullable = true)
 |-- Mat: integer (nullable = true)
 |-- Won: integer (nullable = true)
 |-- Tied: integer (nullable = true)
 |-- NR: integer (nullable = true)
 |-- Runs: integer (nullable = true)
 |-- Wkts: integer (nullable = true)
 |-- Balls: integer (nullable = true)
 |-- Ave: double (nullable = true)
 |-- RPO: double (nullable = true)



## odi_match_results

In [0]:
odi_match_results = odi_match_results.drop('_c0', 'Match_ID', 'Country_ID')

In [0]:
odi_match_results.printSchema()

root
 |-- Result: string (nullable = true)
 |-- Margin: string (nullable = true)
 |-- BR: string (nullable = true)
 |-- Toss: string (nullable = true)
 |-- Bat: string (nullable = true)
 |-- Opposition: string (nullable = true)
 |-- Ground: string (nullable = true)
 |-- Start Date: string (nullable = true)
 |-- Country: string (nullable = true)



## odi_match_totals

In [0]:
odi_match_data = odi_match_totals.drop('_c0', 'Match_ID', 'Country_ID')

In [0]:
column_to_cast = ['Inns', 'Target']
for column in column_to_cast:
    odi_match_data = odi_match_data.withColumn(column, col(column).cast(IntegerType()))

column_to_cast_double = ['Overs', 'RPO']
for columns in column_to_cast_double:
    odi_match_data = odi_match_data.withColumn(columns, col(columns).cast(DoubleType()))

In [0]:
odi_match_data = odi_match_data.withColumnRenamed("Start Date", "Start_Date")

In [0]:
odi_match_data.printSchema()

root
 |-- Score: string (nullable = true)
 |-- Overs: double (nullable = true)
 |-- RPO: double (nullable = true)
 |-- Target: integer (nullable = true)
 |-- Inns: integer (nullable = true)
 |-- Result: string (nullable = true)
 |-- Opposition: string (nullable = true)
 |-- Ground: string (nullable = true)
 |-- Start_Date: string (nullable = true)
 |-- Country: string (nullable = true)



## WC_Players

In [0]:
wc_players = wc_players.withColumn('ID', col('ID').cast(IntegerType()))

In [0]:
wc_players.printSchema()

root
 |-- Player: string (nullable = true)
 |-- ID: integer (nullable = true)
 |-- Country: string (nullable = true)



In [0]:
batsman_data.write.mode('overwrite').option('header', 'true').csv('/mnt/cwc2019analysis/transformed-data/batsman_data')
bowler_data.write.mode('overwrite').option('header', 'true').csv('/mnt/cwc2019analysis/transformed-data/bowler_data')
ground_data.write.mode('overwrite').option('header', 'true').csv('/mnt/cwc2019analysis/transformed-data/ground_data')
odi_match_results.write.mode('overwrite').option('header', 'true').csv('/mnt/cwc2019analysis/transformed-data/odi_match_results')
odi_match_data.write.mode('overwrite').option('header', 'true').csv('/mnt/cwc2019analysis/transformed-data/odi_match_data')
wc_players.write.mode('overwrite').option('header', 'true').csv('/mnt/cwc2019analysis/transformed-data/wc_players')